# Coursera Data Science Capstone Week3

## FIRST SUBMISSION

## Creating the Dataframe

In [92]:
import pandas
import numpy
from IPython.display import display_html
import matplotlib.cm as cm
import matplotlib.colors as colors

In [24]:
url1 = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [25]:
df1 = pandas.read_html(url1,header=0)[0]

In [26]:
#Removing 'Borough' = 'Not assigned'
df2 = df1.drop(df1[df1['Borough']=="Not assigned"].index)
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [27]:
def Combine_Neigh(x):
    return pandas.Series(dict(Neighbourhood=','.join(x['Neighbourhood'])))

In [28]:
#Combining Neighbourhood for (Postcode,Borough) group
df3 = df2.groupby(['Postcode','Borough'],sort=False).apply(Combine_Neigh).reset_index()
df3.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [29]:
#Replacing 'Not assigned' Neighbourhood values with Borough values
for i in range(len(df3)):
    if df3['Neighbourhood'][i] == "Not assigned":
        df3['Neighbourhood'][i] = df3['Borough'][i]
df3.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [30]:
df3.shape

(103, 3)

## SECOND SUBMISSION

In [31]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [32]:
import geocoder # import geocoder
#Geocoder not working. Infinite loop.

In [34]:
#Create dataframe of Geospatial Coordinates
df4 = pandas.read_csv('C:/Users/User/Downloads/Geospatial_Coordinates.csv')
df4.rename(columns={'Postal Code':'Postcode'},inplace=True)
df4.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df5 = pandas.merge(df3,df4,on='Postcode')
df5.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [36]:
df5.shape

(103, 5)

## THIRD SUBMISSION

In [40]:
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [41]:
#Toronto map location
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [42]:
#Create Toronto map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Add markers to map
for lat, lng, borough, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [86]:
df6 = df5[df5['Borough'].str.contains("Toronto")]
df6.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
df6.head()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [87]:
#Create Borough containing 'Toronto' map
map_toronto_new = folium.Map(location=[latitude, longitude], zoom_start=12)

#Add markers to map
for lat, lng, borough, neighborhood in zip(df6['Latitude'], df6['Longitude'], df6['Borough'], df6['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_new)  
    
map_toronto_new

### Explore Neighbourhood using Foursquare

In [53]:
from IPython.display import Image
from IPython.core.display import HTML
import requests
from pandas.io.json import json_normalize

In [56]:
CLIENT_ID = '0LJP2QGV1EXCG5ALHHLTZFP5GZH300JZUEXN1ETTZOL5EE31'
CLIENT_SECRET = '1ZGVSYFAHCDOLCK3X5U2Z555KF0RRXKJOGE315PB5ITNW0ZC'
VERSION = '20191018'
LIMIT = 30
radius = 500

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pandas.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
toronto_venues = getNearbyVenues(names=df6['Neighbourhood'],
                                   latitudes=df6['Latitude'],
                                   longitudes=df6['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [73]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 190 uniques categories.


In [75]:
# one hot encoding
toronto_onehot = pandas.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [77]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in numpy.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pandas.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in numpy.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Steakhouse,Hotel,Asian Restaurant,Gym / Fitness Center,Pizza Place,Smoke Shop,Seafood Restaurant,Burger Joint,Plaza
1,Berczy Park,Farmers Market,Beer Bar,Cocktail Bar,Café,Seafood Restaurant,Jazz Club,Basketball Stadium,Fish Market,Bistro,Museum
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Performing Arts Venue,Climbing Gym,Caribbean Restaurant,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Fast Food Restaurant,Comic Shop,Park,Pizza Place,Butcher,Recording Studio,Burrito Place,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Bar,Sculpture Garden,Plane,Boat or Ferry,Boutique,Airport Service


### K Means Clustering

In [83]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [88]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [89]:
toronto_merged = df6

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Mexican Restaurant,Gym / Fitness Center,Greek Restaurant,Pub,Performing Arts Venue,Historic Site
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,1,Café,Clothing Store,Coffee Shop,Steakhouse,Restaurant,Japanese Restaurant,Sandwich Place,Burger Joint,Plaza,Burrito Place
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Gastropub,Italian Restaurant,Hotel,Restaurant,Hostel,BBQ Joint,Beer Bar,Diner,Poke Place
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Wine Bar,Dance Studio,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Farmers Market,Beer Bar,Cocktail Bar,Café,Seafood Restaurant,Jazz Club,Basketball Stadium,Fish Market,Bistro,Museum


In [94]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = numpy.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(numpy.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'],toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 1st Cluster

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Bakery,Park,Breakfast Spot,Mexican Restaurant,Gym / Fitness Center,Greek Restaurant,Pub,Performing Arts Venue,Historic Site
15,Downtown Toronto,0,Coffee Shop,Gastropub,Italian Restaurant,Hotel,Restaurant,Hostel,BBQ Joint,Beer Bar,Diner,Poke Place
24,Downtown Toronto,0,Coffee Shop,Spa,Italian Restaurant,Café,Bubble Tea Shop,Comic Shop,Bar,Portuguese Restaurant,Japanese Restaurant,Sushi Restaurant
42,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Deli / Bodega,Museum,Beer Bar,Pub,Pizza Place,Gym / Fitness Center,Steakhouse
48,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Gastropub,Deli / Bodega,Gym / Fitness Center,Bakery,Seafood Restaurant,Japanese Restaurant,Pub
86,Central Toronto,0,Coffee Shop,Pub,Fried Chicken Joint,Light Rail Station,Liquor Store,Restaurant,Sports Bar,Supermarket,Sushi Restaurant,Bagel Shop
97,Downtown Toronto,0,Café,Coffee Shop,Deli / Bodega,Restaurant,Steakhouse,Food Court,Japanese Restaurant,Gastropub,Bakery,Pizza Place


### 2nd Cluster

In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,1,Café,Clothing Store,Coffee Shop,Steakhouse,Restaurant,Japanese Restaurant,Sandwich Place,Burger Joint,Plaza,Burrito Place
20,Downtown Toronto,1,Farmers Market,Beer Bar,Cocktail Bar,Café,Seafood Restaurant,Jazz Club,Basketball Stadium,Fish Market,Bistro,Museum
25,Downtown Toronto,1,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Diner,Restaurant,Baby Store,Nightclub,Convenience Store
30,Downtown Toronto,1,Café,Steakhouse,Hotel,Asian Restaurant,Gym / Fitness Center,Pizza Place,Smoke Shop,Seafood Restaurant,Burger Joint,Plaza
31,West Toronto,1,Pharmacy,Bakery,Supermarket,Middle Eastern Restaurant,Music Venue,Park,Pizza Place,Café,Brewery,Bar
36,Downtown Toronto,1,Park,Hotel,Café,Plaza,Basketball Stadium,New American Restaurant,Roof Deck,Salad Place,Japanese Restaurant,Italian Restaurant
37,West Toronto,1,Bar,Asian Restaurant,Vietnamese Restaurant,Pizza Place,Wine Bar,New American Restaurant,Men's Store,Mac & Cheese Joint,Korean Restaurant,Japanese Restaurant
41,East Toronto,1,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Coffee Shop,Cosmetics Shop,Pizza Place,Dessert Shop,Pub,Diner
43,West Toronto,1,Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Performing Arts Venue,Climbing Gym,Caribbean Restaurant,Restaurant,Burrito Place
47,East Toronto,1,Park,Sushi Restaurant,Ice Cream Shop,Brewery,Burger Joint,Burrito Place,Liquor Store,Board Shop,Fast Food Restaurant,Italian Restaurant


### 3rd Cluster

In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,2,Bus Line,Park,Swim School,Wine Bar,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store


### 4th Cluster

In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,3,Home Service,Garden,Ice Cream Shop,Wine Bar,Deli / Bodega,Falafel Restaurant,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant


### 5th Cluster

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,4,Health Food Store,Trail,Pub,Wine Bar,Dance Studio,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run
68,Central Toronto,4,Park,Trail,Jewelry Store,Sushi Restaurant,Wine Bar,Deli / Bodega,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant
83,Central Toronto,4,Gym,Playground,Trail,Tennis Court,Dance Studio,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run
91,Downtown Toronto,4,Park,Playground,Trail,Building,Deli / Bodega,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run
